In [1]:
import pandas as pd
data_dir = "/kaggle/input/image-matching-challenge-2025"
train_labels = pd.read_csv("/kaggle/input/image-matching-challenge-2025/train_labels.csv")
train_labels

,dataset,scene,image,rotation_matrix,translation_vector
0,imc2023_haiper,fountain,fountain_image_116.png,0.122655949;0.947713775;-0.294608417;0.1226706...,0.093771314;-0.803560988;2.062001533
1,imc2023_haiper,fountain,fountain_image_108.png,0.474305910;0.359108654;-0.803787832;0.2888416...,0.358946647;-0.797557548;1.910906929
2,imc2023_haiper,fountain,fountain_image_101.png,0.565115476;-0.138485064;-0.813305838;0.506678...,0.146922468;-0.981392596;2.009002852
3,imc2023_haiper,fountain,fountain_image_082.png,-0.308320392;-0.794654112;0.522937261;0.948141...,0.206413831;-1.174321103;3.667167680
4,imc2023_haiper,fountain,fountain_image_071.png,-0.569002830;-0.103808175;0.815757098;0.778745...,-0.015140892;-1.334052012;3.488936597
...,...,...,...,...,...
1940,stairs,stairs_split_2,stairs_split_2_1710453733751.png,0.961762441;-0.187990401;0.199179859;-0.177691...,-0.112850000;-3.521750000;-2.859750000
1941,stairs,stairs_split_2,stairs_split_2_1710453759963.png,0.237960308;0.580896704;-0.778417569;0.4077886...,-0.490768000;-3.064140000;3.008420000
1942,stairs,stairs_split_2,stairs_split_2_1710453805788.png,0.309067298;0.541767194;-0.781642957;0.4038963...,-0.572757000;0.885835000;4.987270000
1943,stairs,stairs_split_2,stairs_split_2_1710453765165.png,0.301920210;0.609614467;-0.732949103;0.5007116...,-0.135613000;-1.832910000;1.598790000


In [ ]:
!pip install umap-learn
!pip install hdbscan
!pip install pycolmap

In [2]:
import os
import torch
def pick_images(image_dir, datasets=[]):
    image_paths = []
    if len(datasets) == 0:
        df = train_labels
    else:
        df = train_labels[train_labels["dataset"].isin(datasets)]
    for _, row in df.iterrows():
        path = os.path.join(image_dir, row["dataset"], row["image"])
        image_paths.append(path)
    return image_paths

image_dir = f"{data_dir}/train"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
from transformers import CLIPProcessor, CLIPModel
from PIL import Image
import os
import cv2 as cv
import torch
import umap
from hdbscan import HDBSCAN
import gc
    

def cluster_images(image_paths, device):
    # Load Pretrained CLIP
    model_name = "openai/clip-vit-base-patch32"
    clip_model = CLIPModel.from_pretrained(model_name).to(device)
    processor = CLIPProcessor.from_pretrained(model_name)

    def get_image_embeddings(image_path: str):
        image = Image.open(image_path)
    
        with torch.no_grad():
            inputs = processor(images=image, return_tensors="pt", padding=True).to(device)
            return clip_model.get_image_features(**inputs)

    # Get image embeddings using CLIP
    image_embeddings = torch.cat([get_image_embeddings(image_path) for image_path in image_paths])

    # Dimensionality reduction
    umap_model = umap.UMAP(n_components = 5)
    reduced_embeddings = umap_model.fit_transform(image_embeddings.cpu().numpy())

    # Clustering
    clusterer = HDBSCAN(min_cluster_size=3)
    clusterer.fit(reduced_embeddings)

    clusters = clusterer.labels_

    # Cleanup
    del clip_model
    del image_embeddings
    torch.cuda.empty_cache()
    gc.collect()

    return reduced_embeddings, clusters

## Visualize clusters

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
import random

def visualize_clusters(umap_embeddings, cluster_labels, image_paths, num_samples=5):
    """
    Visualizes UMAP-reduced embeddings with HDBSCAN clusters and displays example images from a random cluster.
    
    Args:
        umap_embeddings (numpy.ndarray): 2D UMAP-reduced embeddings (N x 2).
        cluster_labels (numpy.ndarray): Cluster assignments for each image (N,).
        image_paths (list): List of file paths to the images (N,).
        num_samples (int): Number of images to display from a selected cluster.
    """

    unique_clusters = set(cluster_labels)
    palette = sns.color_palette("husl", len(unique_clusters))  # Generate colors for clusters
    
    # Scatter plot of clusters
    plt.figure(figsize=(10, 6))
    for cluster_id in unique_clusters:
        indices = np.where(cluster_labels == cluster_id)[0]
        plt.scatter(umap_embeddings[indices, 0], umap_embeddings[indices, 1], 
                    label=f"Cluster {cluster_id}" if cluster_id != -1 else "Outliers",
                    alpha=0.6, edgecolors='k', s=30, color=palette[cluster_id % len(palette)])
    
    plt.xlabel("UMAP Dimension 1")
    plt.ylabel("UMAP Dimension 2")
    plt.title("UMAP Visualization of Image Clusters")
    plt.legend()
    plt.show()

    # Pick a random cluster (excluding outliers, labeled as -1)
    valid_clusters = [c for c in unique_clusters if c != -1]
    if not valid_clusters:
        print("No valid clusters found.")
        return

    selected_cluster = random.choice(valid_clusters)
    print(f"\nDisplaying {num_samples} images from Cluster {selected_cluster}:")

    # Get image paths from the selected cluster
    cluster_indices = np.where(cluster_labels == selected_cluster)[0]
    sample_indices = random.sample(list(cluster_indices), min(num_samples, len(cluster_indices)))
    sample_images = [image_paths[i] for i in sample_indices]

    # Display images
    fig, axes = plt.subplots(1, len(sample_images), figsize=(15, 5))
    for ax, img_path in zip(axes, sample_images):
        img = Image.open(img_path)
        ax.imshow(img)
        ax.axis("off")
    plt.show()


In [ ]:
# Clustering example on imc2023_haiper
datasets = ["stairs"]
image_paths = pick_images(image_dir, datasets)
reduced_embeddings, clusters = cluster_images(image_paths, device)
visualize_clusters(reduced_embeddings, clusters, image_paths)

## Run COLMAP for Camera Pose Estimation

In [ ]:
import os
import numpy as np
import pandas as pd
import pycolmap
from tqdm import tqdm

def run_colmap_on_cluster(image_paths, output_dir, do_extract=True):
    """Runs COLMAP on a subset of images and returns poses."""
    os.makedirs(output_dir, exist_ok=True)

    database_path = os.path.join(output_dir, "database.db")
    image_dir = os.path.dirname(image_paths[0])

    if do_extract:
        image_list = [os.path.basename(img) for img in image_paths]
        pycolmap.extract_features(
            database_path=database_path,
            image_path=image_dir,
            image_list=image_list,
            camera_mode=pycolmap.CameraMode.AUTO,
            device=pycolmap.Device('auto'),
            sift_options=pycolmap.SiftExtractionOptions(estimate_affine_shape=True),
        )

        pycolmap.match_exhaustive(database_path, device=pycolmap.Device('auto'))

    reconstructions = pycolmap.incremental_mapping(
        database_path=database_path,
        image_path=image_dir,
        output_path=output_dir,
    )

    # Extract poses (R, T) for each image
    poses = {}
    if reconstructions:
        reconstruction = next(iter(reconstructions.values()))
        for image_id, image in reconstruction.images.items():
            img_name = image.name
            rigid3d = image.cam_from_world
            R = rigid3d.rotation.matrix().ravel()
            R = ";".join(map(str, list(R)))
            T = rigid3d.translation
            T = ";".join(map(str, list(T)))
            poses[img_name] = (R, T)
    return poses

def process_clusters(image_paths, cluster_labels, output_dir="colmap_temp", do_extract=True):
    """Processes each cluster with COLMAP and compiles results."""
    results = []

    # Group images by cluster
    clusters = {}
    for idx, label in enumerate(cluster_labels):
        if label not in clusters:
            clusters[label] = []
        clusters[label].append(image_paths[idx])

    # Process each cluster
    for cluster_id, cluster_images in tqdm(clusters.items()):
        if cluster_id == -1:
            # Outliers: No pose estimation
            for img in cluster_images:
                dataset = img.split(os.path.sep)[-2]
                results.append({
                    "dataset": dataset,
                    "scene": "outliers",
                    "image": os.path.basename(img),
                    "rotation_matrix": ";".join(map(str, [np.nan] * 9)),
                    "translation_vector": ";".join(map(str, [np.nan] * 3)),
                })
        else:
            # Run COLMAP on the cluster
            poses = run_colmap_on_cluster(cluster_images, f"{output_dir}/cluster_{cluster_id}", do_extract)

            # Assign results
            for img in cluster_images:
                img_name = os.path.basename(img)
                dataset = img.split(os.path.sep)[-2]
                if img_name in poses:
                    R, T = poses[img_name]
                    results.append({
                        "dataset": dataset,
                        "scene": f"cluster_{cluster_id}",
                        "image": img_name,
                        "rotation_matrix": R,
                        "translation_vector": T,
                    })
                else:
                    # COLMAP failed to register this image
                    results.append({
                        "dataset": dataset,
                        "scene": f"cluster_{cluster_id}",
                        "image": img_name,
                        "rotation_matrix": ";".join(map(str, [np.nan] * 9)),
                        "translation_vector": ";".join(map(str, [np.nan] * 3)),
                    })
    return pd.DataFrame(results)


df_submission = process_clusters(image_paths, clusters, do_extract=False)
df_submission

## Run Fast3R for camera pose estimation

In [ ]:
# Download PyTorch3D from wheel
import sys
import torch
pyt_version_str=torch.__version__.split("+")[0].replace(".", "")
version_str="".join([
    f"py3{sys.version_info.minor}_cu",
    torch.version.cuda.replace(".",""),
    f"_pyt{pyt_version_str}"
])
print(version_str)
!pip install -q iopath
!pip install -q --no-index --no-cache-dir pytorch3d -f https://dl.fbaipublicfiles.com/pytorch3d/packaging/wheels/{version_str}/download.html


In [ ]:
# clone Fast3R repo
!git clone https://github.com/facebookresearch/fast3r
!pip install -q torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu125

# install requirements
!pip install -q -r /kaggle/working/fast3r/requirements.txt

# install fast3r as a package (so you can import fast3r and use it in your own project)
!pip install -q -e /kaggle/working/fast3r

In [ ]:
# Restart to use fast3r module
%load_ext autoreload
%autoreload 2

In [ ]:
# Debugging in case you get ModuleNotFoundError
import os
import pkgutil
import fast3r
import sys
print(sys.path)
print([p for p in sys.path if 'fast3r' in p])
print([k for k in sys.modules if k.startswith('fast3r')])

for sub in pkgutil.iter_modules(fast3r.__path__):
    print(sub)


In [4]:
import numpy as np

def pose_to_data(camera_poses, cluster, cluster_name, dataset):
    data = []
    for pose, image_path in zip(camera_poses, cluster):
        rotation_matrix = pose[:3, :3].flatten()  # shape (9,)
        translation_vector = pose[:3, 3].flatten()  # shape (3,)
        
        rotation_str = ";".join(str(x) for x in rotation_matrix)
        translation_str = ";".join(str(x) for x in translation_vector)
        
        data.append({
            "dataset": dataset,
            "scene": cluster_name,
            "image": os.path.basename(image_path),
            "rotation_matrix": rotation_str,
            "translation_vector": translation_str
        })
    return data

In [17]:
import torch
from fast3r.models.fast3r import Fast3R
from fast3r.models.multiview_dust3r_module import MultiViewDUSt3RLitModule
from fast3r.dust3r.utils.image import load_images
from fast3r.dust3r.inference_multiview import inference
import numpy as np
import gc

def camera_pose_estimation(clusters, dataset, device):
    data = []
    # Load the model from Hugging Face
    model = Fast3R.from_pretrained("jedyang97/Fast3R_ViT_Large_512")
    model.encoder_args["attn_implementation"] = "pytorch_naive"
    model.build_encoder(model.encoder_args)
    model.decoder_args["attn_implementation"] = "pytorch_naive"
    model.build_decoder(model.decoder_args)
    
    model = model.to(device)
    lit_module = MultiViewDUSt3RLitModule.load_for_inference(model)
    
    model.eval()
    lit_module.eval()
    for cluster in clusters:
        if cluster == "outliers":
            # Skip inference for outliers
            for outlier in clusters[cluster]:
                data.append({
                    "dataset": dataset,
                    "scene": cluster,
                    "image": os.path.basename(outlier),
                    "rotation_matrix": ";".join(map(str, [np.nan] * 9)),
                    "translation_vector": ";".join(map(str, [np.nan] * 3))
                })
            continue
            
        # Load images
        images = load_images(clusters[cluster], size=64, verbose=False)
    
        # Run inference
        try:
            output_dict, profiling_info = inference(
                images,
                model,
                device,
                dtype=torch.float16, # Play around with this. original value is float32
                verbose=False,
                profiling=True
            )
        except Exception as e:
            print(e)
            torch.cuda.empty_cache()
            gc.collect()
            continue
        
        # Extract camera poses
        poses_c2w_batch, estimated_focals = MultiViewDUSt3RLitModule.estimate_camera_poses(
            output_dict['preds'],
            niter_PnP=100,
            focal_length_estimation_method='first_view_from_global_head'
        )
        # poses_c2w_batch is a list; the first element contains the estimated poses for each view.
        camera_poses = poses_c2w_batch[0]

        # Record rotation and translation
        data.extend(pose_to_data(camera_poses, clusters[cluster], cluster, dataset))
    
        # Release GPU memory when done
        del poses_c2w_batch
        torch.cuda.empty_cache()
        gc.collect()
        
    return data

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [18]:
# Evaluation on clusters found using CLIP ViT encoder
import pandas as pd
import numpy as np

datasets = list(train_labels["dataset"].unique()) # All datasets in train dir
data = []
for dataset in datasets:
    image_paths = pick_images(image_dir, datasets=[dataset])
    print(f"Dataset: {dataset} with {len(image_paths)} images.")
    print("Clustering images...")
    _, clusters = cluster_images(image_paths, device)
    clusters_dict = {} 
    for path, cluster_name in zip(image_paths, clusters):
        if cluster_name == -1:
            cluster_name = "outliers"
        if cluster_name not in clusters_dict:
            clusters_dict[cluster_name] = []
        clusters_dict[cluster_name].append(path)
    print(f"Found {len(clusters_dict)} clusters.")

    print("Estimating camera poses for each cluster...")
    data.extend(camera_pose_estimation(clusters_dict, dataset, device))
df_submission = pd.DataFrame(data)

Dataset: imc2023_haiper with 54 images.
Clustering images...
Found 3 clusters.
Estimating camera poses for each cluster...
encode_images time: 0.06632351875305176
pos emb time: 0.0007040500640869141
decoder time: 0.042017221450805664
head prepare input time: 0.0006487369537353516
head forward time: 0.024832487106323242
total Fast3R forward time: 0.13488388061523438
encode_images time: 0.06144404411315918
pos emb time: 0.0005242824554443359
decoder time: 0.03298807144165039
head prepare input time: 0.0004923343658447266
head forward time: 0.02007603645324707
total Fast3R forward time: 0.11580419540405273
encode_images time: 0.0626368522644043
pos emb time: 0.0005221366882324219
decoder time: 0.03606057167053223
head prepare input time: 0.0005033016204833984
head forward time: 0.02112555503845215
total Fast3R forward time: 0.12113189697265625
Dataset: imc2023_heritage with 209 images.
Clustering images...
Found 13 clusters.
Estimating camera poses for each cluster...
encode_images time: 

## Save data to CSV and Evaluate

In [19]:
def format_submission(df):
    """Formats the DataFrame into Kaggle's submission format."""
    return df[["dataset", "scene", "image", "rotation_matrix", "translation_vector"]]

# Save to CSV
submission = format_submission(df_submission)
submission.to_csv("train_results.csv", index=False)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [23]:
from IPython.display import HTML
def create_download_link(title = "Download CSV file", filename = "data.csv"):  
    html = '<a href={filename}>{title}</a>'
    html = html.format(title=title,filename=filename)
    return HTML(html)

# create a link to download the dataframe which was saved with .to_csv method
create_download_link(filename='train_results.csv')

In [20]:
import numpy as np
import pandas as pd
from itertools import combinations
from scipy.spatial.distance import cdist

def horn_method(src, dst):
    """Horn's absolute orientation algorithm for similarity transformation"""
    src_centroid = np.mean(src, axis=0)
    dst_centroid = np.mean(dst, axis=0)

    src_centered = src - src_centroid
    dst_centered = dst - dst_centroid

    H = src_centered.T @ dst_centered
    U, _, Vt = np.linalg.svd(H)
    R = Vt.T @ U.T

    if np.linalg.det(R) < 0:
        Vt[-1,:] *= -1
        R = Vt.T @ U.T

    scale = np.linalg.norm(dst_centered) / np.linalg.norm(src_centered)
    t = dst_centroid - scale * R @ src_centroid

    return R, scale, t

def compute_mAA(pred_poses, gt_poses, thresholds):
    """Compute mAA for a single scene using RANSAC-like approach"""
    pred_centers = np.array([-pose[0].T @ pose[1] for pose in pred_poses])
    gt_centers = np.array([-pose[0].T @ pose[1] for pose in gt_poses])
    N = len(pred_centers)
    best_counts = np.zeros(len(thresholds))

    # Try all possible triplets
    for triplet in combinations(range(N), 3):
        try:
            R, scale, t = horn_method(pred_centers[list(triplet)], gt_centers[list(triplet)])
            
            # Apply transformation to all points
            transformed = scale * (R @ pred_centers.T).T + t
            # Count inliers for each threshold
            distances = np.linalg.norm(transformed - gt_centers, axis=1, ord=1)
            
            # Refine transformation with all inliers
            for i, thresh in enumerate(thresholds):
                inliers = np.where(distances < thresh)[0]
                if len(inliers) >= 3:
                    R_refined, scale_refined, t_refined = horn_method(
                        pred_centers[inliers], gt_centers[inliers])
                    transformed_refined = scale_refined * (R_refined @ pred_centers.T).T + t_refined
                    distances_refined = np.linalg.norm(transformed_refined - gt_centers, axis=1)
                    new_inliers = np.where(distances_refined < thresh)[0]

                    if len(new_inliers) > best_counts[i]:
                        best_counts[i] = len(new_inliers)
        except:
            continue

    # Calculate accuracy at each threshold
    accuracies = best_counts / len(pred_centers)
    return np.mean(accuracies)

def parse_pose(row, isPred, cluster):
    """Parse rotation matrix and translation vector from CSV row"""
    if row['scene_pred'] != cluster:
        return None
    try:
        R_col_name = 'rotation_matrix'
        R_col_name += '_pred' if isPred else '_gt'
        T_col_name = 'translation_vector'
        T_col_name += '_pred' if isPred else '_gt'
            
        R = np.array(row[R_col_name].split(';'), dtype=float).reshape(3,3)
        T = np.array(row[T_col_name].split(';'), dtype=float)
        return (R, T)
    except:
        return None

def parse_threshold(row):
    """Parse threshold vector from CSV row"""
    try:
        thresholds = np.array(row['thresholds'].split(';'), dtype=float)
        return thresholds
    except:
        return None

def evaluate_submission(pred_csv, gt_csv, thresholds_csv):
    """Full evaluation pipeline matching competition metrics"""
    pred = pd.read_csv(pred_csv)
    gt = pd.read_csv(gt_csv)
    thresholds = pd.read_csv(thresholds_csv)

    merged = pd.merge(pred, gt,
                    on=['dataset', 'image'],
                    suffixes=('_pred', '_gt'))

    # Group by dataset and scene
    gt_groups = merged.groupby(['dataset', 'scene_gt'])
    pred_groups = merged.groupby(['dataset', 'scene_pred'])

    results = []

    for (dataset, scene), scene_data in gt_groups:
        if scene == 'outliers':
            continue

        # Get ground truth poses
        # gt_poses = [parse_pose(row, False) for _, row in scene_data.iterrows()]
        # gt_poses = [p for p in gt_poses if p is not None]

        gt_thresholds = parse_threshold(thresholds[(thresholds["dataset"]==dataset) & (thresholds["scene"]==scene)].iloc[0])
        
        # if len(gt_poses) < 3:
        #     continue

        # Find best matching predicted cluster
        best_mAA = 0
        best_cluster_size = 0
        best_cluster_overlap = 0
        best_cluster_score = 0
        best_cluster = None

        for (pred_ds, pred_cluster), pred_data in pred_groups:
            if pred_ds != dataset or pred_cluster == 'outliers':
                continue

            # Get ground truth poses
            gt_poses = [parse_pose(row, False, pred_cluster) for _, row in scene_data.iterrows()]
            gt_poses = [p for p in gt_poses if p is not None]

            # Get predicted poses
            pred_poses = [parse_pose(row, True, pred_cluster) for _, row in scene_data.iterrows()]
            pred_poses = [p for p in pred_poses if p is not None]

            if len(gt_poses) != len(pred_poses):
                print(f"Error: Expected the number of ground truth cameras ({len(gt_poses)}) to match the number of user selected camers ({len(pred_poses)})")
                continue

            if len(pred_poses) < 3:
                continue

            # Calculate intersection
            common_images = set(scene_data['image']) & set(pred_data['image'])
            cluster_score = len(common_images) / len(pred_data)

            # Calculate mAA
            try:
                mAA = compute_mAA(pred_poses, gt_poses, gt_thresholds)
            except:
                mAA = 0
                
            if mAA > best_mAA or (mAA == best_mAA and cluster_score > best_cluster_score):
                best_mAA = mAA
                best_cluster_score = cluster_score
                best_cluster = pred_cluster
                best_cluster_overlap = len(common_images)
                best_cluster_size = len(pred_data)

        if best_cluster:
            results.append({
                'dataset': dataset,
                'scene': scene,
                'mAA': best_mAA,
                'cluster_overlap': best_cluster_overlap,
                'cluster_size': best_cluster_size,
                'mAA_counts': best_mAA*best_cluster_size
            })

    # Aggregate results
    if not results:
        return {'mean_mAA': 0, 'mean_clustering_score': 0, 'final_score': 0}

    df_results = pd.DataFrame(results)
    df_results_by_dataset = df_results.groupby("dataset")
    agg_cluster = df_results_by_dataset[["mAA_counts", "cluster_overlap", "cluster_size"]].sum()
    agg_cluster_scores = (agg_cluster["cluster_overlap"] / agg_cluster["cluster_size"])
    agg_mAA = (agg_cluster["mAA_counts"] / agg_cluster["cluster_size"])

    output = pd.concat([agg_mAA, agg_cluster_scores], names=["mAA", "cluster_scores"], axis=1)
    output = output.set_axis(["mAA", "cluster_scores"], axis=1)
    output["dataset_scores"] = 2*output["mAA"]*output["cluster_scores"] / (output["mAA"] + output["cluster_scores"])

    return output


thresholds_file = f'{data_dir}/train_thresholds.csv'
labels_file = f'{data_dir}/train_labels.csv'
results = evaluate_submission('train_results.csv', labels_file, thresholds_file)
print(results)
print(f"Final Competition Score: {np.mean(results['dataset_scores'].to_list())*100:.2f}")

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
<ipython-input-20-07fbd6b5fb62>:22: RuntimeWarning: divide by zero encountered in scalar divide
  scale = np.linalg.norm(dst_centered) / np.linalg.norm(src_centered)
<ipython-input-20-07fbd6b5fb62>:23: RuntimeWarning: invalid value encountered in multiply
  t = dst_centroid - scale * R @ src_centroid
<ipython-input-20-07fbd6b5fb62>:23: RuntimeWarning: invalid value encountered in matmul
  t = dst_centroid - scale * R @ src_centroid
<ipython-input-20-07fbd6b5fb62>:40: RuntimeWarning: invalid value encountered in multiply
  transformed = scale * (R @ pred_centers.T).T + t
<ipython-input-20-07fbd6b5fb62>:46: RuntimeWarning: invalid value en

                                        mAA  cluster_scores  dataset_scores
dataset                                                                    
ETs                                0.000000        0.863636        0.000000
amy_gardens                        0.000000        1.000000        0.000000
fbk_vineyard                       0.032895        0.631579        0.062533
imc2023_haiper                     0.000000        0.981481        0.000000
imc2023_heritage                   0.047107        0.887324        0.089465
imc2023_theather_imc2024_church    0.072368        1.000000        0.134969
imc2024_dioscuri_baalshamin        0.016949        1.000000        0.033333
imc2024_lizard_pond                0.015152        0.909091        0.029806
pt_brandenburg_british_buckingham  0.043704        1.000000        0.083747
pt_piazzasanmarco_grandplace       0.022146        0.994048        0.043327
pt_sacrecoeur_trevi_tajmahal       0.000000        1.000000        0.000000
pt_stpeters_